In [2]:
import torch
from torch.linalg import norm

In [ ]:
norm()

In [46]:
import torch
from torch import nn


class ResNetBlock(nn.Module):
    def __init__(self, in_ch, out_ch, H, W, kernel_size, stride=1, **kwargs):
        super(ResNetBlock, self).__init__()
        
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, padding=(kernel_size - 1) // 2, bias=False, **kwargs),
            nn.LayerNorm([out_ch, H, W]),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=kernel_size, padding=(kernel_size - 1) // 2, stride=stride, bias=False, **kwargs),
            nn.LayerNorm([out_ch, H // stride + H % stride, W // stride + W % stride])
        )
        
        self.conv1x1 = nn.Conv2d(in_ch, out_ch, 1, 1, 0)
        self.pooling = nn.AdaptiveAvgPool2d((H // stride + H % stride, W // stride + W % stride))
        self.final_act = nn.LeakyReLU(0.1, inplace=True)
        
    def forward(self, x):
        x = self.final_act(self.pooling(self.conv1x1(x)) + self.block(x))
        return x


def conv_block_v3(in_ch, out_ch, kernel_size, stride, padding, norm=False, W=None, H=None, activation='leaky_relu', dropout=None, **kwargs):
     return nn.Sequential(
         nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, stride=stride, padding=padding, **kwargs),
         nn.LayerNorm([out_ch, H, W]) if norm else nn.Identity(),
         nn.LeakyReLU(0.1, inplace=True) if 'leaky_relu' else nn.ReLU(inplace=True),
         nn.Dropout2d(dropout) if dropout is not None else nn.Identity()
     )

    
class DensePoseRegressorV6(nn.Module):
    def __init__(self):
        super(DensePoseRegressorV6, self).__init__()
        
        self.decoder = nn.Sequential(
            conv_block_v3(4, 64, 5, 1, 2, True, 160, 120, groups=2, bias=False),
            ResNetBlock(64, 64, 120, 160, 3, 1, groups=2),
            ResNetBlock(64, 64, 120, 160, 3, 2, groups=2),
            ResNetBlock(64, 64, 60, 80, 3, 2, groups=2),
            ResNetBlock(64, 64, 30, 40, 3, 2, groups=2),
            ResNetBlock(64, 64, 15, 20, 3, 2, groups=2),
            ResNetBlock(64, 64, 8, 10, 3, 2, groups=2),    
            nn.Conv2d(64, 64, (4, 5), 1, 0),
        )
        
        self.translation_conv = nn.Conv2d(32, 3, 1, 1, 0)
        self.angle_conv = nn.Conv2d(32, 4, 1, 1, 0)
        
    def forward(self, x):
        x = self.decoder(x)
        return x

In [44]:
x = torch.rand(1, 4, 120, 160)
pr = DensePoseRegressorV6()
pr.cuda()

x = x.cuda()

In [45]:
pr(x)

torch.Size([3, 64, 1, 1])


x = torch.rand(4, 3, 15, 20)
m = nn.Conv2d(3, 3, 5, 2, 2)

x1 = m(x)
print(x1.size())

In [14]:
p = torch.nn.AdaptiveAvgPool2d((8, 10))
p(x).shape

torch.Size([4, 3, 8, 10])